This Python script simulates the dynamics of charged particles under relativistic conditions using the Pygame library. It is designed to visualize and explore concepts such as electromagnetic fields, synchrotron radiation, and the effects of relativity on particle interactions. Here's a breakdown of what the script does:

### Imports and Constants
- **Pygame** is used for graphical rendering and interaction.
- **Numpy** is utilized for numerical operations, especially in vector and matrix computations.
- Constants like `WIDTH`, `HEIGHT`, `SPEED_OF_LIGHT`, and `ELEMENTARY_CHARGE` are defined for use in simulations.

### Synchrotron Function
- `synchrotron_function(x)`: Approximates a function related to synchrotron radiation, which particles emit when they are accelerated in magnetic fields.

### Particle Class
- Represents charged particles with properties like position (`x`, `y`, `z`), `charge`, `mass`, and `velocity`.
- `update`: Updates particle's state based on electromagnetic fields and other particles. It calculates forces from given electric and magnetic fields and updates the particle's velocity and position considering relativistic effects (like mass increase at speeds close to the speed of light and time dilation).
- `synchrotron_radiation_spectrum` and `synchrotron_radiation_power`: Calculate the spectrum and power of synchrotron radiation emitted by the particle.
- `retarded_position` and `retarded_fields`: Compute the position and electromagnetic fields of the particle at previous times, accounting for the finite speed of light (this is part of what makes the simulation 'relativistic').
- `relativistic_mass` and `time_dilation`: Compute the relativistic mass and time dilation effects based on the particle’s velocity.

### ParticleManager Class
- Manages a collection of `Particle` instances.
- `add_particle`: Adds a new particle to the simulation.
- `draw`: Draws all particles on the Pygame screen.
- `update_interactions`: Updates interactions between all particles, considering retarded fields, which simulate the delay in the effect of one particle's field on another due to the finite speed of light.
- `update`: Updates all particles’ states and interactions between them.

### Field Functions
- `electric_field` and `magnetic_field`: Functions to compute the electric and magnetic fields at any point in space based on predefined distributions or configurations.

### Simulation Loop
- Initializes Pygame and sets up the simulation environment.
- Enters a loop where it processes user events (like quitting or adding particles with mouse clicks), updates the state of the simulation (particle movements and interactions), and renders the updated state to the screen.
- Uses Pygame’s drawing functions to visualize particles and possibly field lines or intensity.

### Key Concepts Illustrated by the Script
- **Relativistic Effects**: The script accounts for effects such as time dilation and relativistic mass increase, essential when simulating particles moving at speeds close to the speed of light.
- **Electromagnetic Interactions**: It computes interactions between particles based on electromagnetic fields, including those generated by the particles themselves, considering the retardation effect due to the finite speed of light.
- **Synchrotron Radiation**: It calculates and possibly visualizes the radiation emitted by charged particles when accelerated, which is significant in many high-energy physics contexts.

Overall, this script provides a platform for visualizing and understanding the complex dynamics involved in relativistic particle interactions, including electromagnetic forces and radiation. It serves as an educational tool for illustrating concepts from electromagnetism and relativistic physics.

In [1]:
# Acceleration  + Momentum 
# V 9.1.6 Relativistic Particle Simulation ZOOM
 #conservation of energy
# This version of the simulation focuses on the dynamic behavior of relativistic 
# particles under various electromagnetic influences. Below is an overview of the
# key features and physical phenomena modeled in the code:

### Particle Characteristics
# Lorentz Force**: Accurately models the electromagnetic force on charged particles, 
# integrating both electric and magnetic field effects.

### Energy Loss and Radiation
# Synchrotron Radiation**: Simulates energy dissipation due to synchrotron radiation 
# for particles in magnetic fields. This involves computing critical frequencies and the 
# associated power radiated.

### Advanced Electromagnetic Effects
# Landau-Lifshitz Radiation Reaction**: Incorporates a simplified version of the 
# Landau-Lifshitz expression to account for the radiation reaction force alongside 
# the Lorentz force, enhancing the realism in modeling particle dynamics.

# Retarded Time Effects**: Takes into account the finite speed of light for interactions 
# between particles, employing the concept of retarded time to calculate forces more accurately.

# Liénard-Wiechert Potentials**: Utilizes electric forces between moving particles 
# to model retarded effects based on Liénard-Wiechert potentials.

# Biot-Savart Law**: Applies the Biot-Savart law for calculating magnetic forces 
# between moving particles, further refining the simulation of retarded effects.

### Numerical Integration and Relativistic Modeling
# Runge-Kutta 4th Order Integration (RK4)**: Employs the RK4 method for robust 
# numerical integration, updating particle positions and velocities over time 
# considering the applied forces.

# Relativistic Effects**: Ensures the simulation remains physically accurate 
# at high velocities by incorporating relativistic corrections, notably through 
# the use of the Lorentz factor (gamma) in force, acceleration, and energy calculations.

### Simulation Enhancements
# Lorentz Transformation of Fields**: Includes transformations between the 
# laboratory frame of reference and the particle's frame of reference, allowing 
# for detailed analysis of relativistic effects.
#  Adaptive Time Stepping**: Adjusts the time step dynamically based on the 
# Simulation conditions to optimize accuracy and computational efficiency.
# Relativistic Equations of Motion**: Fully integrates relativistic equations 
# of motion into the simulation framework, ensuring comprehensive coverage of relativistic dynamics. 
import uuid
import pygame 
import copy
import colorsys
class Particle:      
    def __init__(self, initial_pos, initial_velocity, 
                 charge, mass, force_method, radiation_reaction, velocity_method,
                 electric_field_function, magnetic_field_function,letter):          
        self.dt = 1e-9
        #self.time = 0                
        self.force_method = lambda pos, vel: force_method(self, pos, vel)
        self.rad_method = radiation_reaction   
        self.velocity_method = velocity_method        
        self.electric_function = electric_field_function
        self.magnetic_function = magnetic_field_function         
        
        self.total_E_field = np.array([0.0, 0.0, 0.0])
        self.total_B_field = np.array([0.0, 0.0, 0.0]) 
         
        self.charge, self.mass = charge, mass 
        self.position = initial_pos
        self.velocity = em_equations.limit_speed(initial_velocity)
        self.acc = np.array([0.0, 0.0, 0.0])    
        self.gamma = em_equations.Gamma(self.velocity)        
        self.total_energy = self.gamma * self.mass * (em_equations.c ** 2)  
        self.energy_loss = 0  
        self.sync_freq = 0 
        
        self.trajectory=1000
        self.dt_traj  = [self.dt]
        self.pos_traj = [self.position]
        self.vel_traj = [self.velocity]
        self.acc_traj = [self.acc]           
        self.letter = letter 
        self.id = str(uuid.uuid4())  
        self.radius = 5  
        self.color = (0, 0, 0)               
        self.thickness = 0
        self.active = True
        
    def update(self, particles, dt, Q, B ):
        self.dt = dt   
        #self.mass = M * self.f_mass        
        # Calculate base and retarded electric and magnetic fields                
        #retarded_E_field, retarded_B_field = self.retarded_effects(particles, dt)          
        #self.total_E_field =  self.electric_function(self.position, Q)  #+ retarded_E_field
        #self.total_B_field =  self.magnetic_function(self.position, B)  #+ retarded_B_field  

        retarded_E_field, retarded_B_field = self.retarded_effects(particles, dt)

        self.total_E_field = self.electric_function(self.position, Q)
        self.total_B_field = self.magnetic_function(self.position, B)
     

            
        # Check if retarded_E_field and retarded_B_field are valid (not null or inf)
        if not np.isnan(retarded_E_field).any() and not np.isinf(retarded_E_field).any():
            self.total_E_field += retarded_E_field        
        if not np.isnan(retarded_B_field).any() and not np.isinf(retarded_B_field).any():
            self.total_B_field += retarded_B_field

        # Momentum before the update 
        Energy_old = self.gamma * self.mass * (em_equations.c ** 2) 
        P_old = self.gamma*self.mass*self.velocity  
        
        ##########################################################################
        #Energy loss due to radiation          
        v_synch_new, delta_Es, KE_final = em_equations.synchrotron_radiation(self, self.total_B_field, self.velocity, dt) 
        v_rad_new, delta_Er, KE_final = self.rad_method(self, self.total_B_field, self.velocity, dt)         
        #self.velocity = self.update_velocity(v_synch_new, v_rad_new)         
        self.velocity = v_synch_new
        ###########################################################################         
         
        self.position, self.velocity = self.velocity_method(self)          
        self.velocity = em_equations.limit_speed(self.velocity)     
        self.gamma = em_equations.Gamma(self.velocity)
        
        P_new = self.gamma*self.mass*self.velocity
        self.acc = (P_new-P_old)/self.dt 
        
        #self.time+= dt
        #self.energy_loss= delta_Er+delta_Es
        new_total_energy = self.gamma * self.mass * (em_equations.c ** 2)    
        if Energy_old - new_total_energy >0:
            self.energy_loss = Energy_old - new_total_energy      
        else:
            self.energy_loss = 0    
            
        self.update_color_based_on_radiation()      
        
        # Update Trajectory         
        self.step_count = getattr(self, 'step_count', 0) + 1        
        attributes = ('position', 'velocity', 'dt', 'acc')
        for attr, traj in zip(attributes, (self.pos_traj, self.vel_traj, self.dt_traj, self.acc_traj)):
            traj.append(np.copy(getattr(self, attr)))
            if self.step_count <= 100 or len(traj) > self.trajectory:
                traj.pop(0)  
                
    def update_error(self, particles, dt, Q, B ):     
        self.dt = dt
        retarded_E_field, retarded_B_field = self.retarded_effects(particles, dt)          
        #self.total_E_field = self.electric_function(self.position, Q) #+ retarded_E_field
        #self.total_B_field = self.magnetic_function(self.position, B) #+ retarded_B_field   

        self.total_E_field = self.electric_function(self.position, Q)
        self.total_B_field = self.magnetic_function(self.position, B)
        
        # Check if retarded_E_field and retarded_B_field are valid (not null or inf)
        if not np.isnan(retarded_E_field).any() and not np.isinf(retarded_E_field).any():
            self.total_E_field += retarded_E_field
        
        if not np.isnan(retarded_B_field).any() and not np.isinf(retarded_B_field).any():
            self.total_B_field += retarded_B_field
    
        P_old = self.gamma * self.mass * self.velocity
        self.position, self.velocity = self.velocity_method(self)
        self.velocity = em_equations.limit_speed(self.velocity)  
        #print('error',self.velocity)
        gamma = em_equations.Gamma(self.velocity)
        P_new = gamma * self.mass * self.velocity
        self.acc = (P_new - P_old) / self.dt      

    def retarded_effects(self, particles, dt):                      
        total_E, total_B = np.zeros(3), np.zeros(3)    
        # Create a new list of particles excluding those with null velocity or position
        if self.velocity is None or self.position is None:
            return total_E, total_B 
            
        valid_particles = [p for p in particles if not (p.velocity is None or p.position is None)]       
        for p in valid_particles:
            if p.id == self.id:
                continue    
            r, retarded_time = em_equations.calculate_retardation(self, p)
            retarded_position, retarded_velocity, retarded_acc = em_equations.retarded_state(p, retarded_time)
            r_retarded, r_retarded_mag, r_retarded_unit = em_equations.calculate_retarded_distance(self, retarded_position)            
            E = em_equations.calculate_electric_field(p, r_retarded, r_retarded_mag, r_retarded_unit, retarded_velocity, retarded_acc)
            B = em_equations.calculate_magnetic_field(p, r_retarded_mag, r_retarded_unit, retarded_velocity)            
            total_E += E
            total_B += B
            
            if   np.isnan(total_E).any() and not np.isinf(total_E).any():
                print(p.letter,'E')
            if   np.isnan(total_B).any() and not np.isinf(total_B).any():
                print(p.letter,'B')
        return total_E, total_B             

###################################################################################            
    def update_color_based_on_radiation(self):        
        normalized_power = min(self.energy_loss / self.total_energy, 1)
        brightness = np.clip(np.log10((0.7 + 0.3 * normalized_power) * 10), 0, 1)
        normalized_speed = np.clip(np.log10(10 * np.linalg.norm(em_equations.limit_speed(self.velocity)) / em_equations.c), 0, 1)
        hue = 0.3 * (1 - normalized_speed)  
        min_gamma, max_gamma = 1, 30
        normalized_gamma = (self.gamma - min_gamma) / (max_gamma - min_gamma)
        saturation = 0.5 + 0.5 * min(normalized_gamma, 0.99)
        if np.linalg.norm(self.velocity) < em_equations.c:
            self.color = tuple(int(255 * component) for component in colorsys.hsv_to_rgb(hue, saturation, brightness))
        else:
            pass   
    def update_velocity(self, v_synch_new, v_rad_new):
        delta_v_synch = np.linalg.norm(v_synch_new - self.velocity)
        delta_v_rad = np.linalg.norm(v_rad_new - self.velocity)
        B_unit = self.total_B_field / np.linalg.norm(self.total_B_field)
        v_unit = self.velocity / np.linalg.norm(self.velocity)
        perp_dir = np.cross(np.cross(B_unit, v_unit), B_unit)
        perp_dir /= np.linalg.norm(perp_dir)  
        return self.velocity + delta_v_synch * perp_dir + delta_v_rad * v_unit             
    
#############################################################################
#############################################################################

class ParticleManager:
    def __init__(self):
        self.particles = []  # List to hold all particles
        self.dt = 1e-9
        self.ERROR_THRESHOLD = 0.001
        self.MIN_DT = 1e-15
        self.MAX_DT = 1e-9
        self.MIN_INCREASE_FACTOR = 1.05
        self.MAX_DECREASE_FACTOR = 0.75 

    def update(self, electric_field, magnetic_field, Q, B, M):
        # Initial factors set based on class or instance level preferences
        decrease_factor = 0.7
        increase_factor = 1.2
        for particle in self.particles:
            if not particle.active:
                continue  # Skip updating inactive particles           
            error, p_vel = self.estimate_error(particle, electric_field, magnetic_field, Q, B)
            
            # Check if particle velocity is greater than or equal to the speed of light
            while np.linalg.norm(p_vel) >= em_equations.c:
                # Adjust timestep until velocity is smaller than the speed of light
                self.adjust_timestep(error, self.ERROR_THRESHOLD, decrease_factor, increase_factor, self.MIN_DT, self.MAX_DT)
                decrease_factor = max(decrease_factor * 0.8, self.MAX_DECREASE_FACTOR)
                increase_factor = max(increase_factor * 0.975, self.MIN_INCREASE_FACTOR)
                
                # Recompute error and particle velocity with the adjusted timestep
                error, p_vel = self.estimate_error(particle, electric_field, magnetic_field, Q, B)
            
            # Proceed with adjusting timestep based on the error threshold
            if error != 0:
                self.adjust_timestep(error, self.ERROR_THRESHOLD, decrease_factor, increase_factor, self.MIN_DT, self.MAX_DT)
                decrease_factor = max(decrease_factor * 0.9, self.MAX_DECREASE_FACTOR)
                increase_factor = max(increase_factor * 0.975, self.MIN_INCREASE_FACTOR)        
        
        self.update_particles(Q, B, M)

    

 
        
    def add_particle(self, initial_pos, initial_velocity, charge, mass, dt, 
                     force_method, radiation_reaction, velocity_method,
                     electric_field, magnetic_field,letter):
        self.particles.append(Particle(initial_pos, initial_velocity, charge, mass, 
                                       force_method, radiation_reaction, velocity_method, 
                                       electric_field, magnetic_field, letter))   
        
    def update_particles(self, Q, B, M):
        for particle in self.particles:
            particle.update( self.particles, self.dt, Q, B )        
            
    def estimate_error(self,p, electric_field, magnetic_field, Q, B):
        # Perform one step with Δt
        original_state = self.save_state(p)
        p.update_error( self.particles, self.dt , Q, B)        
        final_state_single_step = self.save_state(p)
        self.restore_state(original_state,p)
        
        # Perform two steps with Δt/2
        p.update_error( self.particles, self.dt/2, Q, B )
        p.update_error( self.particles, self.dt/2, Q, B )         
        final_state_two_steps = self.save_state(p)
        self.restore_state(original_state,p)
        
        # Calculate the error between the two approaches
        error =np.linalg.norm(np.array(final_state_single_step['position'])-np.array(final_state_two_steps['position']))
        #error = 0 
        return error, final_state_single_step['position']

    def adjust_timestep(self, error, threshold, decrease_factor, increase_factor, min_dt, max_dt):
        if error > threshold:
            self.dt *= decrease_factor
        else:
            self.dt = min(self.dt * increase_factor, max_dt)
        self.dt = max(min_dt, min(self.dt, max_dt))            
    
    def save_state(self, p):
        return {
            'position': p.position.copy(),  # Copy the position array
            'velocity': p.velocity.copy(),  # Copy the velocity array
            'acc': p.acc.copy()             # Copy the acceleration array
        }
    
    def restore_state(self, state, p):
        p.position = copy.deepcopy(state['position'])
        p.velocity = copy.deepcopy(state['velocity'])
        p.acc = copy.deepcopy(state['acc'])         

############################################################################# 

def electric_field(position,Q_F):
    EPSILON_0 = 8.8541878128e-12
    x, y, z =position
    # _chaos  Constants     
    k = 1 / (4 * np.pi * EPSILON_0)
    Q = Q_F# 1e-14# Q_F Q#0e-13#-2#1e17*ELEMENTARY_CHARGE
    source_x, source_y, source_z = 401, 300, 0  
    sigma_x, sigma_y, sigma_z = 160, 120, 50  # Standard deviations of the distribution
    Lx, Ly = WIDTH, HEIGHT  # Dimensions of the simulation domain
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))
    perturbation = 0.1 * (np.sin(2 * np.pi * 2 * x / Lx - z) + 0.5 * np.cos(3 * np.pi * 6 * y / Ly - 2*z) + 0.2 * np.sin(x*y*z))
    field_magnitude = gaussian #* (1 + 2*perturbation)
    field_x = k * field_magnitude * dx / (sigma_x**2 )
    field_y = k * field_magnitude * dy / (sigma_y**2 )
    field_z = k * field_magnitude * dz / (sigma_z**2 )
    return np.array([field_x, field_y, field_z])
         
def magnetic_field(position, B):
    x, y, z =position
    B_x = 0#np.zeros_like(x)
    B_y =  0#np.zeros_like(y)
    B_z = B#np.ones_like(z) * 1e-1 # Example: constant magnetic field along the z-axis    
    return np.array([B_x, B_y, B_z ])

def electric_field_point(x, y, z, charge_pos):
    EPSILON_0 = 8.8541878128e-12 
    k = 1 / (4 * np.pi * EPSILON_0)
    Q =0.5  # Charge of the point charge
    source_x, source_y, source_z = charge_pos  # Unpack the charge position
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    distance_squared = dx**2 + dy**2 + dz**2
    field_magnitude = k * Q / distance_squared  # Coulomb's Law
    # Ensure we don't divide by zero
    distance = np.sqrt(distance_squared)
    distance = np.maximum(distance, epsilon)  # A small value to avoid division by zero    
    field_x = field_magnitude * dx / distance
    field_y = field_magnitude * dy / distance
    field_z = field_magnitude * dz / distance
    return np.array([field_x, field_y, field_z])

def electric_field_theoretical(x, y, z):
    k = 8.9875517923e9  # Coulomb's constant
    r = np.sqrt(x**2 + y**2 + z**2)
    E_magnitude = k * q / r**2  # Replace q with the source charge
    E_direction = np.array([x, y, z]) / r  # Unit vector in the direction of r
    return E_magnitude * E_direction

def electric_field_acc(x, y, z):
    EPSILON_0 = 8.8541878128e-12 
    # Constants for the accelerator
    center_x, center_y, center_z = 400, 300, 0  # Central point of the accelerator
    Q = 1  # Charge creating the field, simplified assumption
    k = 1 / (4 * np.pi * EPSILON_0)  # Coulomb's constant
    # Compute differences from the center and distances
    dx, dy, dz = x - center_x, y - center_y, z - center_z
    distance = np.sqrt(dx**2 + dy**2 + dz**2) + epsilon  # Add epsilon to avoid division by zero
    # Compute radial and angular directions
    radial_direction = np.array([dx, dy, dz]) / distance
    angular_direction = np.array([-dy, dx, np.zeros_like(dz)]) / (np.sqrt(dx**2 + dy**2) + epsilon)  
    # Compute the electric field
    electric_field = Q * k * radial_direction  # Simplified; adjust as needed for your model
    return electric_field


def magnetic_field_g(x, y, z):
    MAGNETIC_FIELD_STRENGTH = 1
    ACCELERATOR_LENGTH = 100
    # Convert scalar inputs to arrays
    if isinstance(z, (int, float)):
        z = np.array([z])
    # Define the magnetic field for guiding the particles
    mask = (0 <= z) & (z < ACCELERATOR_LENGTH)
    B = np.zeros((3, *z.shape))
    B[1, mask] = MAGNETIC_FIELD_STRENGTH
    # If the input was scalar, return a 1D array
    if B.shape[1] == 1:
        return B[:, 0]
    return B

def electric_field_spiral (x, y, z):
    # Constants
    k = 8.9875517873681764e9
    Q = 1
    source_x, source_y, source_z = 400, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 50, 50, 50  # Standard deviations of the distribution
    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    # Compute the radial distance and angle in the xy-plane
    r = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)
    # Compute the spiral factor
    spiral_factor = np.exp(1j * (r / 20))
    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))
    # Compute the electric field components
    field_x = k * gaussian * (dx / (sigma_x**2 + epsilon) * spiral_factor.real - dy / (sigma_y**2 + epsilon) * spiral_factor.imag)
    field_y = k * gaussian * (dy / (sigma_y**2 + epsilon) * spiral_factor.real + dx / (sigma_x**2 + epsilon) * spiral_factor.imag)
    field_z = k * gaussian * dz / (sigma_z**2 + epsilon)
    return np.array([field_x, field_y, field_z])
 

#############################################################################

from visual import VisualizationManager
from physics import ElectromagneticEquations
import numpy as np

em_equations = ElectromagneticEquations()
particle_manager = ParticleManager()
epsilon = np.finfo(np.float64).eps
ERROR_THRESHOLD = 0.001
num_prticle = 9
GRID_SPACING = 20
WIDTH, HEIGHT = 800, 600 
dt = 1e-9
B = 0.05 
Q_F= 1e19 * em_equations.ELEMENTARY_CHARGE  

charge_mult=1
mas_multiply =100
  
p_initial_velocity = np.array([1e6, 0.0, 0.0])
e_initial_velocity = np.array([2e8, 0e8, 0.0])
 
visual_manager = VisualizationManager(WIDTH, HEIGHT, GRID_SPACING,                                        
                                      num_prticle, particle_manager)

visual_manager.run_simulation(mas_multiply, charge_mult, e_initial_velocity, p_initial_velocity,   
                              ERROR_THRESHOLD, dt, B, Q_F, electric_field, magnetic_field)
 

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.5)


In [ ]:
#comparing the simulation to the analytical solution of a moving charge partical on a constant magnetic field. 
# Set up the initial conditions
x0, y0, z0 = 0, 0, 0
v0 = 1e6  # Initial velocity magnitude (m/s)
alpha = 0 #np.pi / 4  # Angle between initial velocity and x-y plane (45 degrees)
q = 1 #1.60217663e-19  # Charge of the particle (C)
m = 1e-7 #1e-28 #9.10938356e-31  # Mass of the particle (kg)
dt = 1e-9  # Time step (s)
B = 0.1   # Magnetic field strength (T)
 


# Calculate the cyclotron frequency
omega = (q * B) / m

# Set the initial velocity components
vx0 =  v0 * np.cos(alpha)
vy0 = v0 * np.sin(alpha)
vz0 =0

# Create a particle with the initial conditions
particle = Particle(x0, y0, z0, q, m)
particle.velocity = np.array([vx0, vy0, vz0])
particle.dt= dt

# Define the magnetic field function
def magnetic_field_function(x, y, z):
    return np.array([0, 0, B])

# Define the electric field function
def electric_field_function(x, y, z):
    return np.array([0, 0, 0])

# Simulate the particle motion for a given time
time_steps = 6000
positions = []
positions.append([particle.x, particle.y, particle.z])
for _ in range(time_steps):
    particle.update(electric_field_function, magnetic_field_function, [])
    positions.append([particle.x, particle.y, particle.z])

# Convert positions to numpy array
positions = np.array(positions)

# Calculate analytical positions
t = np.arange(0, time_steps * dt, dt)
x_analytical = (vx0 / omega) * np.sin(omega * t)
y_analytical = (vx0 / omega) * (np.cos(omega * t) - 1)
z_analytical = vz0 * t

# Plot the simulated and analytical trajectories
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], label='Simulated')
ax.plot(x_analytical[0:], y_analytical[0:], z_analytical[0:], label='Analytical')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.show()